# OCR 모델 이용해 text detection & text recognition 작업 수행

In [1]:
# import torch
# torch.cuda.is_available()
# torch.cuda.get_device_name(0)

In [2]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import glob

import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import paddleocr
from paddleocr import PaddleOCR, draw_ocr
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## 데이터 구조 확인하기

### 데이터 목록이 제대로 있는지 확인하는 코드

# ! 다음 내용을 이용해서 각자 데이터 다운받은것 갯수 확인 부탁드려요
# 각자 코드 돌릴 때 데이터가 다를 경우 추후 오류날 가능성 있음

In [3]:
# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
contents = ['R.Free/', 'O.Form/']
json_files_list = []
png_list = []

for background in backgrounds:
    for content in contents:
        tmp = []
        path = basic_path + background + content
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
        json_files_list.append(tmp)
        print('{}.json: {}개'.format(path, len(tmp)))
        
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.png'), os.listdir(path))))
        png_list.append(tmp)
        print('{}.png: {}개'.format(path, len(tmp)))
        print()

data/HW_OCR/4.Validation/P.Paper/R.Free/.json: 35535개
data/HW_OCR/4.Validation/P.Paper/R.Free/.png: 35535개

data/HW_OCR/4.Validation/P.Paper/O.Form/.json: 43170개
data/HW_OCR/4.Validation/P.Paper/O.Form/.png: 43170개

data/HW_OCR/4.Validation/T.Tablet/R.Free/.json: 4789개
data/HW_OCR/4.Validation/T.Tablet/R.Free/.png: 4789개

data/HW_OCR/4.Validation/T.Tablet/O.Form/.json: 12525개
data/HW_OCR/4.Validation/T.Tablet/O.Form/.png: 12525개



In [4]:
P1 = len(json_files_list[0]) + len(json_files_list[1])
T1 = len(json_files_list[2]) + len(json_files_list[3])
print('갖고 있는 json 데이터 목록')
print(P1, T1, P1+T1)
print('데이터 사이트 - 내가 갖고 있는 데이터 목록')
print(87205 - P1, 19314 - T1, 106519 - (P1+T1))

print()

P2 = len(png_list[0]) + len(png_list[1])
T2 = len(png_list[2]) + len(png_list[3])
print('갖고 있는 png 데이터 목록')
print(P2, T2, P2+T2)
print('데이터 사이트 - 내가 갖고 있는 데이터 목록')
print(87205 - P2, 19314 - T2, 106519 - (P2+T2))

갖고 있는 json 데이터 목록
78705 17314 96019
데이터 사이트 - 내가 갖고 있는 데이터 목록
8500 2000 10500

갖고 있는 png 데이터 목록
78705 17314 96019
데이터 사이트 - 내가 갖고 있는 데이터 목록
8500 2000 10500


In [5]:
path = 'data/HW_OCR/4.Validation/P.Paper/R.Free/'
file = 'IMG_OCR_53_4PR_21045.json'
with open(path + file, 'r', encoding='UTF8') as f:
    data = json.load(f)
pprint.pprint(data)

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
             'identifier': 'IMG_OCR_53',
             'label_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'name': '대용량 손글씨 데이터셋',
             'src_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'type': 1},
 'Images': {'acquistion_location': '자체',
            'application_field': '기타',
            'background': 0,
            'data_captured': '2021.09.06',
            'height': 3482,
            'identifier': 'IMG_OCR_53_4PR_21045',
            'media_type': 0,
            'pen_color': 'red',
            'pen_type': 0,
            'type': 'png',
            'width': 2469,
            'writer_age': 47,
            'writer_sex': 1,
            'written_content': 1},
 'bbox': [{'data': '021045',
           'id': 1,
           'x': [1857, 1857, 2262, 2262],
           'y': [153, 248, 153, 248]},
          {'data': '경상북도',
           'id': 2,
           'x': [347, 347, 746, 74

### 결과 데이터 구성 요소

- label에서 가져올 정보
    - id
        - Images > identifier
    - image
        - Images > identifer + '.' + type
        - 두 요소를 .으로 이어 붙어야 한다.
    - age
        - Images > writer_age
    - sex
        - Images > writer_sex
    - background
        - Images > background
        - 종이: 0
        - 타블렛: 1
    - text_origin
        - bbox > data
    - bbox_origin
        - bbox > data, id, x, y 순서로 정렬
        - x와 y에 존재하는 숫자는 좌측 하부, 좌측 상부, 우측 하부, 우측 상부 순서로 정렬되어 있음
        - x1, y1, x2, y2 순서로 정리 필요
            - x2 = x1 + w
            - y2 = y1 + h

<br>

- OCR 모델에서 가져올 정보
    - text_90
        - 판별 score가 90점이 넘는 글씨의 텍스트 값
    - bbox_90
        - 판별 score가 90점이 넘는 글씨의 bbox 값
    - text_80
        - 판별 score가 80점이 넘는 글씨의 텍스트 값
    - bbox_80
        - 판별 score가 80점이 넘는 글씨의 bbox 값

In [6]:
# 결과 데이터 컬럼 및 df 생성

col = ['id', 'image', 'age', 'sex', 'background', 'text_origin', 'bbox_origin', 'text_90', 'bbox_90', 'text_80', 'bbox_80']

ocr_df = pd.DataFrame(columns = col)
ocr_df

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80


In [7]:
# JSON 파일 로드 함수
def load_json(json_path):
    try:
        encoder = 'utf-8'
        with open(json_path, 'r', encoding = encoder) as f:
            data = json.load(f)
    except:
        try:
            encoder = 'cp949'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
            
        except:
            encoder = 'euc-kr'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
    
    return data

# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
content = 'R.Free/'
json_files_list = []

for background in backgrounds:
    tmp = []
    path = basic_path + background + content
    tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
    json_files_list += tmp
    
print(len(json_files_list))


40324


In [8]:
def json2ocrdf(json_files_list):
    for i, json_file in enumerate(json_files_list):
        if i % 100 == 0:
            print(i, end = ' ')
        # JSON 파일 경로
        json_path = json_file
        data = load_json(json_path)

        image_path = 'data/' + data['Dataset']['src_path'] + data['Images']['identifier'] + '.' + data['Images']['type']
#         print(image_path)

        # 각 단어의 바운딩 박스와 텍스트 정보
        texts_list = []
        bounding_box_list = []

        # 전처리된 이미지 리스트
        for word in data['bbox']:
            text = word['data']
            texts_list.append(text)
#             print(text)
            # x, y, x+w, y+h
            bounding_box = [word['x'][0], word['y'][0], word['x'][2], word['y'][1]]
            bounding_box_list.append(bounding_box)
#             print(bounding_box)

        ocr_id = data['Images']['identifier']
        ocr_image = image_path
        ocr_age = data['Images']['writer_age']
        ocr_sex = data['Images']['writer_sex']
        ocr_background = data['Images']['background']
        ocr_text_origin = texts_list
        ocr_bbox_origin = bounding_box_list
        ocr_text_90 = None
        ocr_bbox_90 = None
        ocr_text_80 = None
        ocr_bbox_80 = None

        ocr_df.loc[len(ocr_df)] = [ocr_id, ocr_image, ocr_age, ocr_sex, ocr_background, ocr_text_origin, ocr_bbox_origin,
                                   None, None, None, None]
        

In [9]:
json2ocrdf(json_files_list[0:1000])

0 100 200 300 400 500 600 700 800 900 

In [10]:
ocr_df.to_csv('result_df_sample.csv', index = False)

In [11]:
ocr_df

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80
0,IMG_OCR_53_4PR_04358,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,"[004358, 경기도, 대전광역시, 충청북도, 울산광역시, 정읍시, 군위군, 부산...","[[1941, 138, 2190, 223], [339, 1016, 566, 1134...",None,None,None,None
1,IMG_OCR_53_4PR_04359,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,"[004359, 대전광역시, 서울특별시, 제주특별자치도, 경상남도, 미추홀구, 김포...","[[1907, 145, 2188, 230], [308, 997, 683, 1128]...",None,None,None,None
2,IMG_OCR_53_4PR_04360,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,"[004360, 경상남도, 세종특별자치시, 충청남도, 부산광역시, 예산군, 달서구,...","[[1951, 144, 2216, 226], [312, 1003, 662, 1142...",None,None,None,None
3,IMG_OCR_53_4PR_04361,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,"[004361, 서울특별시, 광주광역시, 부산광역시, 세종특별자치시, 양평군, 관악...","[[1894, 140, 2131, 218], [369, 976, 699, 1117]...",None,None,None,None
4,IMG_OCR_53_4PR_04362,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,"[004362, 광주광역시, 충청남도, 대구광역시, 울산광역시, 고양시, 일산동구,...","[[1903, 143, 2171, 229], [352, 1007, 730, 1153...",None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
995,IMG_OCR_53_4PR_05458,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005458, 서울특별시, 세종특별자치시, 경상북도, 울산광역시, 경산시, 창원시...","[[1850, 147, 2104, 226], [357, 1027, 660, 1162...",None,None,None,None
996,IMG_OCR_53_4PR_05459,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005459, 전라북도, 광주광역시, 충청북도, 경기도, 여수시, 사하구, 강서구...","[[1856, 132, 2251, 224], [340, 1015, 735, 1150...",None,None,None,None
997,IMG_OCR_53_4PR_05460,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005460, 제주특별자치도, 충청북도, 경상북도, 세종특별자치시, 의성군, 관악...","[[1874, 141, 2106, 229], [316, 1024, 738, 1147...",None,None,None,None
998,IMG_OCR_53_4PR_05461,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[부산광역시, 대전광역시, 울산광역시, 전라북도, 안산시, 계룡시, 안양시, 창원시...","[[382, 1012, 679, 1134], [863, 1022, 1170, 114...",None,None,None,None


In [12]:
json2ocrdf(json_files_list[1000:10000])

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 

In [13]:
ocr_df.tail()

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80
9995,IMG_OCR_53_4PR_15630,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,38,0,0,"[015630, 전라남도, 서울특별시, 충청북도, 경상남도, 의왕시, 옹진군, 곡성...","[[1889, 131, 2151, 210], [382, 985, 664, 1101]...",None,None,None,None
9996,IMG_OCR_53_4PR_15631,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,38,0,0,"[015631, 경기도, 광주광역시, 충청북도, 대구광역시, 성남시, 수정구, 곡성...","[[1882, 118, 2167, 226], [334, 984, 698, 1132]...",None,None,None,None
9997,IMG_OCR_53_4PR_15632,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,38,0,0,"[015632, 전라북도, 전라남도, 울산광역시, 경상북도, 진주시, 구례군, 당진...","[[1876, 121, 2135, 213], [352, 1001, 638, 1119...",None,None,None,None
9998,IMG_OCR_53_4PR_15633,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,38,0,0,"[015633, 충청북도, 경기도, 전라남도, 서울특별시, 곡성군, 아산시, 창원시...","[[1852, 121, 2115, 213], [366, 981, 638, 1119]...",None,None,None,None
9999,IMG_OCR_53_4PR_15634,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,38,0,0,"[015634, 전라북도, 경기도, 전라남도, 제주특별자치도, 의왕시, 춘천시, 영...","[[1832, 128, 2166, 213], [337, 990, 675, 1111]...",None,None,None,None


In [14]:
json2ocrdf(json_files_list[10000:20000])

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 

In [15]:
ocr_df.tail()

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80
19995,IMG_OCR_53_4PR_26766,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,27,0,0,"[026766, 전라북도, 충청북도, 경상남도, 경상북도, 고양시, 일산서구, 영광...","[[1818, 131, 2053, 222], [373, 996, 655, 1120]...",None,None,None,None
19996,IMG_OCR_53_4PR_26767,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,27,0,0,"[026767, 경상북도, 대전광역시, 서울특별시, 울산광역시, 성주군, 강진군, ...","[[1785, 129, 2027, 211], [310, 1002, 617, 1128...",None,None,None,None
19997,IMG_OCR_53_4PR_26768,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,27,0,0,"[026768, 광주광역시, 충청남도, 경상남도, 충청북도, 보성군, 경주시, 완주...","[[1812, 118, 2073, 213], [358, 977, 685, 1121]...",None,None,None,None
19998,IMG_OCR_53_4PR_26769,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,27,0,0,"[026769, 부산광역시, 충청남도, 제주특별자치도, 전라북도, 북구, 부여군, ...","[[1825, 127, 2012, 214], [358, 1017, 727, 1135...",None,None,None,None
19999,IMG_OCR_53_4PR_26770,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,27,0,0,"[026770, 제주특별자치도, 전라남도, 서울특별시, 대전광역시, 포항시, 가평군...","[[1813, 137, 2019, 209], [339, 1021, 728, 1119...",None,None,None,None


In [16]:
json2ocrdf(json_files_list[20000:30000])

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 

In [17]:
ocr_df.tail()

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80
29995,IMG_OCR_53_4PR_60067,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,19,1,0,"[14, 30, 16, 30, 아이들, 책읽어주기, 아이들, 이야기, 들어주기, 야...","[[498, 73, 620, 173], [660, 63, 775, 158], [92...",None,None,None,None
29996,IMG_OCR_53_4PR_60068,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,19,1,0,"[화목지역, 아동센터의, 첫날, 센터에, 오는, 아동들과의, 첫대면, 선입견과, 벽...","[[528, 66, 866, 197], [948, 58, 1381, 187], [1...",None,None,None,None
29997,IMG_OCR_53_4PR_60069,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,19,1,0,"[9, 30, 10, 00, 도서관, 다용도실, 정리, 정돈, 10, 00, 11,...","[[554, 48, 636, 156], [666, 67, 795, 161], [93...",None,None,None,None
29998,IMG_OCR_53_4PR_60070,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,19,1,0,"[13, 아이들과, 이야기도, 하고, 책도, 읽어줌, 지체장애, 중, 3, 그림단어...","[[505, 0, 628, 71], [776, 110, 1100, 218], [11...",None,None,None,None
29999,IMG_OCR_53_4PR_60071,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,19,1,0,"[자율적으로, 학습을, 해나가는, 아이들을, 보면서, 환하게, 웃는, 나를, 봅니다...","[[506, 82, 902, 209], [995, 73, 1219, 202], [1...",None,None,None,None


In [18]:
json2ocrdf(json_files_list[30000:])

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 

In [19]:
ocr_df.tail()

,id,image,age,sex,background,text_origin,bbox_origin,text_90,bbox_90,text_80,bbox_80
40319,IMG_OCR_53_4TR_19310,data/HW_OCR/4.Validation/T.Tablet/R.Free/IMG_O...,49,0,1,"[수출, 대책, 방문하다, 광장, 사다, 제한되다, 자부심, 아무래도, 원고, 싫다...","[[320, 504, 424, 564], [565, 489, 686, 579], [...",None,None,None,None
40320,IMG_OCR_53_4TR_19311,data/HW_OCR/4.Validation/T.Tablet/R.Free/IMG_O...,49,0,1,"[면하다, 울음, 녀석, 만족스럽다, 옛날이야기, 쇼, 토론, 역할, 명, 구멍, ...","[[290, 499, 412, 553], [564, 491, 636, 552], [...",None,None,None,None
40321,IMG_OCR_53_4TR_19312,data/HW_OCR/4.Validation/T.Tablet/R.Free/IMG_O...,49,0,1,"[먼저, 감정평가사무소, 개설, 및, 휴, 폐업, 신고의무를, 폐지하기로, 했다, ...","[[322, 500, 401, 568], [490, 500, 721, 570], [...",None,None,None,None
40322,IMG_OCR_53_4TR_19313,data/HW_OCR/4.Validation/T.Tablet/R.Free/IMG_O...,49,0,1,"[올해, 상반기, 농식품, 수출액이, 역대, 최고, 실적을, 달성했다, 국토부는, ...","[[320, 509, 394, 561], [554, 512, 651, 556], [...",None,None,None,None
40323,IMG_OCR_53_4TR_19314,data/HW_OCR/4.Validation/T.Tablet/R.Free/IMG_O...,49,0,1,"[내라, 팔리다, 감사, 요리하다, 반장, 휴지통, 아래층, 입력하다, 제품, 별다...","[[322, 520, 418, 556], [543, 518, 677, 559], [...",None,None,None,None


In [21]:
ocr_df.to_csv('result_df.csv', index = False)

# OCR_paddle

## OCR 모델 적용하여 

In [ ]:
ocr_model = PaddleOCR(lang = 'korean')

img_path = '2ali6i.jpg'
result = ocr_model.ocr(img_path, cls = False)

In [ ]:
res = result[0]
print(type(res))
boxes = [res[i][0] for i in range(len(result[0]))] # 
texts = [res[i][1][0] for i in range(len(result[0]))]
scores = [float(res[i][1][1]) for i in range(len(result[0]))]

ocr_text_90 = []
ocr_bbox_90 = []
ocr_text_80 = []
ocr_bbox_80 = []

# bbox가 직각사각형이 아닐 수도 있다.
# 관련된 부분 전처리 필요

for t, s, b in zip(texts, scores, boxes):
    print(t, s, b)
    if s >= 0.8:
        ocr_text_80.append(t)
        ocr_bbox_80.append(b)
        
        if s >= 0.9:
            ocr_text_90.append(t)
            ocr_bbox_90.append(b)

print()
print(ocr_text_90, ocr_bbox_90)
print()
print(ocr_text_80, ocr_bbox_90)

In [ ]:
for t, s, b in zip(texts, scores, boxes):
    print(b)

In [ ]:
ocr_bbox_80[0]

### 전처리 작업
- 손글씨만 활용해야 함
- 문서내에 인쇄된 글씨 부분은 제외해야 함
- 반복적으로 나오는 부분을 데이터 확인해 미리 만들어 둠
- 목록에서 제거하기로 함

In [ ]:
# 자간 데이터 코드 :

# def is_handwritten(text, max_char_spacing=5):
#     char_positions = [ord(c) for c in text]
#     spacings = [char_positions[i+1] - char_positions[i] for i in range(len(char_positions) - 1)]
#     average_spacing = sum(spacings) / len(spacings) if spacings else 0
#     return average_spacing > max_char_spacing

# # OCR 결과에서 손글씨만 필터링

# handwritten_texts = [text for text in ocr_results if is_handwritten(text)]

In [ ]:
# # predict > texts
# # json > texts_list

# print('< 예측 | 실제 > ')
# for predict, real in zip(texts, texts_list):
#     print('{} | {}'.format(predict, real))

In [ ]:
#for res in result:
print(boxes[0])
print(texts[0])
print(scores[0])#, Text: {res[1]}

# Specifying font path for draw_ocr method
font_path = 'C:/Users/UserK/ds_study/DL_proj/PaddleOCR-main/doc/fonts/korean.TTF'

img_path = 'C:/Users/UserK/ds_study/DL_proj/2ali6i.jpg'

# imports image
img = cv2.imread(img_path) 

# reorders the color channelsa
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

# Visualize our image and detections
# resizing display area
plt.figure(figsize = (65, 65))

# draw annotations on image
annotated = draw_ocr(img, boxes, texts, scores, font_path = font_path) 

# show the image using matplotlib
plt.imshow(annotated)